In [24]:
import numpy as np
import scipy.sparse
import math



def BinSketch(X, k):
    '''Sketch generation using BinSketch: This method takes a binary matrix X of size nxd, and generates its sketch matrix of size nxk, where k<<d. 
        Parameters:
           X  is a sparse array (NumPy array or can be scipy sparse matrix) of shape (n, d),
              where n is the number of samples and d is feature dimension
           K is the  reduced dimension value (must be a positive integer)
            
        Returns:
          Sketch matrix of X of size nxk-- scipy sparse csr_matrix of shape (n, k)
    '''


    d = X.shape[1]
    random_map = np.array([], int)
    M = np.arange(k)
    for i in range(d//k):
        np.random.shuffle(M)
        random_map = np.concatenate((random_map, M))
    np.random.shuffle(M)
    random_map = np.concatenate((random_map, M[0:d%k]))
    np.random.shuffle(random_map)

    new_X = np.zeros((X.shape[0], k))
    for i in range(k):
        index = np.where(random_map == i)[0]
        new_X[:,i] = np.max(X[:,index].A, axis = 1)
        
    return scipy.sparse.csr_matrix(new_X)

def BinSketch_Sparsity_Estimate(a):
    '''This method takes a low-dimensional (sketch) binary vectors a obtained from the BinSketch method and estimate the sparsity for full-dimesnion version.
        Parameters:
            Low-dimensional (sketch) binary vector a
        Returns:
            An estimate of the sparsity for full-dimensional version of a
    '''
    N = a.shape[1]
    spar_a = a.nnz   #nnz is scipy sparse matrix method to find number of nonzero entries
    
    if spar_a/N < 1:
        spar_est_a = round(math.log(1-(spar_a/N))/math.log(1- 1/N))
    else:
        spar_est_a = spar_a
    return spar_est_a


def BinSketch_IP_Estimate(a, b):
    ''' This method takes two low-dimensional (sketch) binary vectors a and b obtained from the BinSketch method and estimate the Inner product between their full-dimensional version.
        Parameters:
            Low-dimensional (sketch) binary vectors a, b  obtained from BinSketch 

        Returns:
            An estimate of the Inner product between full-dimensional vectors corresponding to a and b
    '''
    N = a.shape[1]
    spar_est_a = BinSketch_Sparsity_Estimate(a)
    spar_est_b = BinSketch_Sparsity_Estimate(b)
  
    IP = int(a.dot(b.T)[0,0])
    val = (1 - 1/N)**(spar_est_a) + (1 - 1/N)**(spar_est_b) + (IP/N) - 1
    if  val > 0:
        Bin_IP_Est  = round(spar_est_a + spar_est_b - (math.log(val) / math.log(1 - 1/N)))
        if Bin_IP_Est < 0:
            Bin_IP_Est = IP
    else:
        Bin_IP_Est = IP
    return Bin_IP_Est
   
def BinSketch_Ham_Estimate(a, b):
    '''This method takes two low-dimensional (sketch) binary vectors a and b obtained from the BinSketch method and estimate the hamming distance between their full-dimensional version.
        Parameters:
            binary vectors a, b 
        Returns:
            An estimate of the Hamming distance between  full-dimensional vectors corresponding to a and b

    '''
    spar_est_a = BinSketch_Sparsity_Estimate(a)
    spar_est_b = BinSketch_Sparsity_Estimate(b)
    Bin_IP_Est = BinSketch_IP_Estimate(a, b)
    
    Bin_Ham_Est = spar_est_a + spar_est_b - 2*Bin_IP_Est
    
    return Bin_Ham_Est


def BinSketch_JS_Estimate(a, b):
    '''This method takes two low-dimensional (sketch) binary vectors a and b obtained from the BinSketch method and estimate the Jaccard similarity between their full-dimensional version.
        Parameters:
            Low-dimensional (sketch) binary vectors a, b  obtained from BinSketch 

        Returns:
            An estimate of the Jaccard similarity between  full-dimensional vectors corresponding to a and b
    '''
    spar_est_a = BinSketch_Sparsity_Estimate(a)
    spar_est_b = BinSketch_Sparsity_Estimate(b)
    Bin_IP_Est = BinSketch_IP_Estimate(a, b)
    Bin_Ham_Est = BinSketch_Ham_Estimate(a,b)
    
    Bin_JS_Est = Bin_IP_Est/(Bin_Ham_Est + Bin_IP_Est)
    
    return Bin_JS_Est

def BinSketch_CS_Estimate(a, b):
    '''This method takes two low-dimensional (sketch) binary vectors a and b obtained from the BinSketch method and estimate the Cosine similarity between their full-dimensional version.
        Parameters:
            Low-dimensional (sketch) binary vectors a, b  obtained from BinSketch
        Returns:
           An estimate of the Cosine similarity between  full-dimensional vectors corresponding to a and b
    '''
    spar_est_a = BinSketch_Sparsity_Estimate(a)
    spar_est_b = BinSketch_Sparsity_Estimate(b)
    Bin_IP_Est = BinSketch_IP_Estimate(a, b)
    Bin_Ham_Est = BinSketch_Ham_Estimate(a,b)
    
    Bin_CS_Est = Bin_IP_Est/(spar_est_a * spar_est_b)**(0.5)
    
    return Bin_CS_Est

In [25]:
#Example
X = scipy.sparse.load_npz('Sample.npz')
print('Shape of actual matrix:', X.shape)


new_X = BinSketch(X, 1000)
print('Shape of compressed sktech matrix:', new_X.shape)

Shape of actual matrix: (100, 102660)
Shape of compressed sktech matrix: (100, 1000)


In [26]:
'''Hamming estimate'''
a = X[0,:]
b = X[1,:]

spar_a = a.nnz  #nnz is scipy sparse  array method to find number of non-zero entries
spar_b = b.nnz

#Finding Inner produt(IP), Hamming distance, Cosine similiarity, Jaccard Similarity between a and b
Actual_IP      = a.dot(b.T)[0,0]
Actual_Ham     = spar_a + spar_b - 2*Actual_IP
Actual_Jaccard = Actual_IP/(Actual_Ham + Actual_IP)
Actual_Cosine  = Actual_IP/(spar_a + spar_b)*(0.5)

print('Sparsity of a is:', spar_a)
print('Sparsity of b is:', spar_b)
print('Inner product of a and b is :', Actual_IP)
print('Hamming distance between a and b is :', Actual_Ham)
print('Jaccard similarity between a and b is :', Actual_Jaccard)
print('Cosine similarity between a and b is:', Actual_Cosine)

print('\nEstimation of similarity measures for a and b using BinSketch\n')
# a_new and b_new corresponds to compressed sketch of a and b
a_new = new_X[0,:]
b_new = new_X[1,:]

spar_est_a = BinSketch_Sparsity_Estimate(a_new)
spar_est_b = BinSketch_Sparsity_Estimate(b_new)
IP_Est     = BinSketch_IP_Estimate(a_new, b_new)
Ham_Est    = BinSketch_Ham_Estimate(a_new, b_new)
JS_Est     = BinSketch_JS_Estimate(a_new, b_new)
CS_Est     = BinSketch_CS_Estimate(a_new, b_new)

print('Sparsity estimate of a:', spar_est_a)
print('Sparsity estimate of a:', spar_est_b)
print('IP estimate of a and b using BinSketch is :', IP_Est)
print('Hamming estimate of a and b  using BinSketch is :', Ham_Est)
print('Jaccard similarity estimate of a and b  using BinSketch is :', JS_Est)
print('Cosine similarity estimate of a and b  using BinSketch is :', CS_Est)

Sparsity of a is: 128
Sparsity of b is: 108
Inner product of a and b is : 6
Hamming distance between a and b is : 224
Jaccard similarity between a and b is : 0.02608695652173913
Cosine similarity between a and b is: 0.012711864406779662

Estimation of similarity measures for a and b using BinSketch

Sparsity estimate of a: 130
Sparsity estimate of a: 109
IP estimate of a and b using BinSketch is : 8
Hamming estimate of a and b  using BinSketch is : 223
Jaccard similarity estimate of a and b  using BinSketch is : 0.03463203463203463
Cosine similarity estimate of a and b  using BinSketch is : 0.06720553796450181
